# Amazon Fine Food Reviews Analysis

Data set Source: https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews data set consists of reviews of fine foods.

- Total Data points: 568,454
- Total Unique Users: 256,059
- Total Products: 74,258
- Time Preiod: Oct 1999 - Oct 2012
- Number of Attributes/Columns in data: 10

Attributes/Features:
1. Id
2. ProductId
3. UserId
4. ProfileName
5. HelpfulnessNumerator
6. HelpfulnessDenominator
7. Score
8. Time
9. Summary
10. Text

## Objective:

Given a review, determine whether the review is positive or negative.

For this project we will be using our 'Score' column as the basis for our class label. Where a rating of 4 or higher will be categorized as a positive review and a rating of 2 or less will be categorized as a negative review.
A rating of 3 in Neutral and will be ignored.

In [1]:
import sqlite3
import pandas as pd
import seaborn as sbn
import matplotlib.pyplot as plt
import numpy as np
import string
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

In [2]:
conn = sqlite3.connect('database.sqlite')

In [3]:
raw_filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""",conn)

In [4]:
raw_filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
raw_filtered_data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [6]:
def make_label(x):
    if x > 3:
        return 'Positive'
    else: 
        return 'Negative'

In [7]:
raw_filtered_data['Score'] = raw_filtered_data['Score'].map(make_label)
raw_filtered_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
525809,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,Positive,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
525810,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,Negative,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
525811,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,Positive,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
525812,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,Positive,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


## Data Cleaning

In [8]:
display = pd.read_sql_query("""
SELECT userID, productID, count(userID) as Common_Count
FROM Reviews
WHERE Score != 3 
GROUP BY userID, ProfileName , Summary , Text, Time
HAVING count(userID) > 1
ORDER BY ProductID
""", conn)
display

,UserId,ProductId,Common_Count
0,A1033RWNZWEMR5,B00004RYGX,3
1,A1048CYU0OV4O8,B00004RYGX,3
2,A10L8O1ZMUIMR2,B00004RYGX,3
3,A11QN7NDE3AROH,B00004RYGX,3
4,A157XTSMJH9XA4,B00004RYGX,3
...,...,...,...
53004,A8Y0K5BEEW2LP,B009NTCO4O,5
53005,AVFP6Y641X6CZ,B009NTCO4O,5
53006,A3RTG00GE3NXQU,B009OM66IU,4
53007,ABE4NMHQOP4W4,B009OM66IU,4


In [9]:
sorted_filtered_data = raw_filtered_data.sort_values('ProductId',  ascending = True, )

In [10]:
sorted_filtered_data.shape

(525814, 10)

In [11]:
print(f"Shape Before: {sorted_filtered_data.shape}")
final_filtered_data = sorted_filtered_data.drop_duplicates(subset=["UserId", "Summary", "Text", "Time"], keep='first', inplace=False)
print(f"Shape After:  {final_filtered_data.shape}")

Shape Before: (525814, 10)
Shape After:  (365293, 10)


In [12]:
print(f"Percentage of common reviews{final_filtered_data.shape[0]*100/sorted_filtered_data.shape[0]}")

Percentage of common reviews69.47190451376342


In [13]:
display = pd.read_sql_query("""
SELECT * FROM Reviews
WHERE Score != 3 AND HelpfulnessNumerator > HelpfulnessDenominator
""",conn)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
1,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...


In [14]:
final_filtered_data = final_filtered_data[final_filtered_data.HelpfulnessNumerator <= final_filtered_data.HelpfulnessDenominator]

In [15]:
final_filtered_data.shape

(365291, 10)

In [16]:
final_filtered_data['Score'].value_counts()

Score
Positive    307932
Negative     57359
Name: count, dtype: int64

## Bag of Words

In [17]:
count_vect = CountVectorizer() # To compute Bag of Words representation(from scikit-learn) 
final_counts = count_vect.fit_transform(final_filtered_data['Text'].values)

In [18]:
final_counts

<365291x115281 sparse matrix of type '<class 'numpy.int64'>'
	with 19418882 stored elements in Compressed Sparse Row format>

## Text Preprocessing:
### - Stemming
### - Stop-words removal
### - Lemmatization

### Steps
- Remove html tags (eg: `<br>`, `<a href>`)
- Remove punctuation marks and characters like (. , #)
- Check if word is made of english letters and not junk(alpha-numeric eg: a87sd87)
- Check for length greater than 2 (There are no adjectives in less than 2 letters)
- Convert to lowercase 
- Remove Stopwords
- Perform stemming (Snowball Stemming) to remove similar words(eg: taste, tasty,tasteful) 

In [19]:
import re
import string 
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [20]:
stop_words = {'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves'}
snow = nltk.stem.SnowballStemmer('english')

In [21]:
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,' ',sentence)
    return cleantext

def clean_punctuation(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned

In [22]:
final_filtered_data['Text'].values[0]

"this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college"

In [23]:
i=0
all_positive_words = []
all_negative_words = []
final_strings = []
s = ''
for sent in final_filtered_data['Text']:
    filtered_sentence = []
    sent= cleanhtml(sent)
    for w in sent.split():
        cleaned_word = clean_punctuation(w)
        if((cleaned_word.isalpha()) & (len(cleaned_word)>2)):
            if cleaned_word.lower() not in stop_words :
                s = (snow.stem(cleaned_word.lower())).encode('utf8')
                filtered_sentence.append(s)
                if(final_filtered_data['Score'].values)[i] =='Positive':
                    all_positive_words.append(s)
                else:
                    all_negative_words.append(s)
    strl = b" ".join(filtered_sentence) 
    final_strings.append(strl)  
    i+=1

## Add cleaned text to the dataframe

In [24]:
final_filtered_data['CleanedText'] = final_strings

In [25]:
final_filtered_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh recit car dr...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,Positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak watch realli rosi movi inco...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,Positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,Positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,b'great littl book read nice rhythm well good ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,Positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,b'book poetri month goe month cute littl poem ...
...,...,...,...,...,...,...,...,...,...,...,...
178145,193174,B009RSR8HO,A4P6AN2L435PV,romarc,0,0,Positive,1350432000,LOVE!! LOVE!!,"LOVE, LOVE this sweetener!! I use it in all m...",b'love sweeten use unsweeten flavor unsweeten ...
173675,188389,B009SF0TN6,A1L0GWGRK4BYPT,Bety Robinson,0,0,Positive,1350518400,Amazing!! Great sauce for everything!,You have to try this sauce to believe it! It s...,b'tri sauc believ start littl sweet honey tast...
204727,221795,B009SR4OQ2,A32A6X5KCP7ARG,sicamar,1,1,Positive,1350604800,Awesome Taste,I bought this Hazelnut Paste (Nocciola Spread)...,b'bought hazelnut past local shop palm tast ex...
5259,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,Positive,1351209600,DELICIOUS,Purchased this product at a local store in NY ...,b'purchas product local store kid love quick e...


## Create a new database with cleaned text for future use.

In [26]:
conn = sqlite3.connect('final.sqlite')
c = conn.cursor()
conn.text_factory = str
final_filtered_data.to_sql('Reviews', conn,schema = None, if_exists = 'replace')

365291

In [27]:
final_filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""",conn)

## Observing the Cleaned text

In [28]:
final_filtered_data

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh recit car dr...
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,Positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak watch realli rosi movi inco...
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,Positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,Positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,b'great littl book read nice rhythm well good ...
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,Positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,b'book poetri month goe month cute littl poem ...
...,...,...,...,...,...,...,...,...,...,...,...,...
365286,178145,193174,B009RSR8HO,A4P6AN2L435PV,romarc,0,0,Positive,1350432000,LOVE!! LOVE!!,"LOVE, LOVE this sweetener!! I use it in all m...",b'love sweeten use unsweeten flavor unsweeten ...
365287,173675,188389,B009SF0TN6,A1L0GWGRK4BYPT,Bety Robinson,0,0,Positive,1350518400,Amazing!! Great sauce for everything!,You have to try this sauce to believe it! It s...,b'tri sauc believ start littl sweet honey tast...
365288,204727,221795,B009SR4OQ2,A32A6X5KCP7ARG,sicamar,1,1,Positive,1350604800,Awesome Taste,I bought this Hazelnut Paste (Nocciola Spread)...,b'bought hazelnut past local shop palm tast ex...
365289,5259,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,Positive,1351209600,DELICIOUS,Purchased this product at a local store in NY ...,b'purchas product local store kid love quick e...


### We now have words categorized into positive and negative.

In [29]:
all_positive_words[::1000000]

[b'witti',
 b'purchas',
 b'youv',
 b'easili',
 b'eat',
 b'mani',
 b'old',
 b'shelf',
 b'assum',
 b'announc']

In [30]:
all_negative_words[::100000]

[b'one',
 b'mix',
 b'send',
 b'relas',
 b'nasti',
 b'buy',
 b'though',
 b'state',
 b'cigarett',
 b'process',
 b'hour',
 b'agre',
 b'box',
 b'assum',
 b'coffe',
 b'contact',
 b'eat',
 b'cooki',
 b'howev',
 b'get']

### Make a Frequency Distribution for the TF-IDF calculation.

In [31]:
freq_dist_positive = nltk.FreqDist(all_positive_words)
freq_dist_negative = nltk.FreqDist(all_negative_words)
freq_dist_positive

FreqDist({b'like': 133703, b'tast': 104680, b'love': 104420, b'use': 97996, b'great': 88889, b'good': 88643, b'one': 86499, b'tri': 77071, b'flavor': 74857, b'make': 72695, ...})

In [32]:
print("Most common positive words: ",freq_dist_positive.most_common()[:10])
print("Most common negative words: ",freq_dist_negative.most_common()[:10])

Most common positive words:  [(b'like', 133703), (b'tast', 104680), (b'love', 104420), (b'use', 97996), (b'great', 88889), (b'good', 88643), (b'one', 86499), (b'tri', 77071), (b'flavor', 74857), (b'make', 72695)]
Most common negative words:  [(b'like', 31019), (b'tast', 28727), (b'product', 20698), (b'one', 17873), (b'would', 17864), (b'tri', 16426), (b'use', 14469), (b'get', 13513), (b'buy', 13283), (b'flavor', 13238)]


## Observation
- Most common positive and negative words are common
- Using Bi-grams could fix this issue

# TF- IDF

In [35]:
tf_idf_vect = TfidfVectorizer(ngram_range = (1,2))
final_tf_idf = tf_idf_vect.fit_transform(final_filtered_data['CleanedText'].values)

In [36]:
final_tf_idf

<365291x2591202 sparse matrix of type '<class 'numpy.float64'>'
	with 20643824 stored elements in Compressed Sparse Row format>

In [37]:
features = tf_idf_vect.get_feature_names_out()
len(features)

2591202

In [38]:
features[100000:100010]

array(['applesauc dont', 'applesauc dri', 'applesauc easili',
       'applesauc eat', 'applesauc egg', 'applesauc enjoy',
       'applesauc even', 'applesauc ever', 'applesauc everi',
       'applesauc everytim'], dtype=object)

In [39]:
print(len(final_tf_idf[3,:].toarray()[0]))

2591202


### Create Function to return uni-gram, bi-gram with top tf_idf value with corresponding ranks

- Since we obtained uncertain results by using uni-gram we also take bi-gram into consideration. This can be commonly seen in our dataset and in real-life that words often provide more meaning with the context.
- It can be seen that "like" is the most common word in both positive and negative words but when we read the actual reviews with the whole context we find that the meaning of words is often altered by the word preceding or succeeding the given word.
- eg: the words "do like", "don't like" will completely lose their meaning as soon as we use uni-gram.
- therefore we will be using bi-gram which provides a good balance between the size of dataset produced and the quality of output.

In [40]:
def top_tfidf_features(row,features,top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_features(final_tf_idf[1,:].toarray()[0], features,25)

In [41]:
top_tfidf

,feature,tfidf
0,cover paperback,0.206787
1,sendak watch,0.206787
2,rosi movi,0.206787
3,movi incorpor,0.206787
4,grew read,0.206787
5,flimsi take,0.206787
6,keep page,0.206787
7,paperback seem,0.206787
8,read sendak,0.206787
9,incorpor love,0.200394


# Applying KNN

In this step we will look into training knn by testing K-NN for multiple values to obtain the optimal value of K
- Check k for multiple values until most optimal value is obtained.

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from collections import Counter

In [43]:
r_tr, r_test, y_tr, y_test = train_test_split(final_tf_idf,final_filtered_data['Score'].values,train_size = 0.8,random_state = 42) 
r_train, r_cv, y_train, y_cv = train_test_split(r_tr,y_tr,train_size = 0.75 ,random_state = 42) 

In [ ]:
for i in range(1,24,2):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(r_train,y_train)
    prediction = knn.predict(r_cv)
    accuracy = accuracy_score(y_cv,prediction,normalize = True)*float(100)
    print(f"Accuracy for k = {i} : {accuracy}")

Accuracy for k = 1 : 84.52188672014016
Accuracy for k = 3 : 84.29877631470886
Accuracy for k = 5 : 84.2686632538531
Accuracy for k = 7 : 84.2495003969449
Accuracy for k = 9 : 84.59853814777301
Accuracy for k = 11 : 84.83396753264529
Accuracy for k = 13 : 84.95852610254866
Accuracy for k = 15 : 85.05297161159626
Accuracy for k = 17 : 85.09266610090613
Accuracy for k = 19 : 85.15699854909798
Accuracy for k = 21 : 85.18848038544718
Accuracy for k = 23 : 85.18848038544718


In [42]:
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(r_train,y_train)
prediction = knn.predict(r_cv)
accuracy = accuracy_score(y_cv,prediction,normalize = True)*float(100)
print(f"Accuracy for k = {i} : {accuracy}")

Accuracy for k = 365291 : 85.09266610090613


In [43]:
knn = KNeighborsClassifier(n_neighbors=200)
knn.fit(r_train,y_train)
prediction = knn.predict(r_cv)
accuracy = accuracy_score(y_cv,prediction,normalize = True)*float(100)
print(f"Accuracy for k = {i} : {accuracy}")

Accuracy for k = 365291 : 84.89829998083714


In [44]:
knn = KNeighborsClassifier(n_neighbors=1000)
knn.fit(r_train,y_train)
prediction = knn.predict(r_cv)
accuracy = accuracy_score(y_cv,prediction,normalize = True)*float(100)
print(f"Accuracy for k = {i} : {accuracy}")

Accuracy for k = 365291 : 84.46850447589587


In [46]:
knn = KNeighborsClassifier(n_neighbors=30)
knn.fit(r_train,y_train)
prediction = knn.predict(r_cv)
accuracy = accuracy_score(y_cv,prediction,normalize = True)*float(100)
print(f"Accuracy for 30 = {i} : {accuracy}")

Accuracy for 30 = 365291 : 85.3212516083112


In [46]:
knn = KNeighborsClassifier(n_neighbors=40)
knn.fit(r_train,y_train)
prediction = knn.predict(r_cv)
accuracy = accuracy_score(y_cv,prediction,normalize = True)*float(100)
print(f"Accuracy for 40 = {i} : {accuracy}")

Accuracy for 40 = 365291 : 85.30893262887021


In [47]:
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(r_train,y_train)
prediction = knn.predict(r_cv)
accuracy = accuracy_score(y_cv,prediction,normalize = True)*float(100)
print(f"Accuracy for 50 = {i} : {accuracy}")

Accuracy for 50 = 365291 : 85.25691916011937


# Testing the model
 - In the training phase of the model we learned that k = 40 is an optimal value with which the model acquires an accuacy of 85.3% approximately for the cross-validation dataset.

In [44]:
knn_optimal = KNeighborsClassifier(n_neighbors = 40)
knn_optimal.fit(r_train,y_train)
prediction = knn_optimal.predict(r_test)
accuracy = accuracy_score(y_test,prediction,normalize = True)*float(100)
print(f"Accuracy of the model: {accuracy}")

Accuracy of the model: 85.67459176829686


# Conclusion
In summary, my analysis using K-NN demonstrated that the model could effectively classify Amazon Fine Food Reviews into 'Positive' and 'Negative' categories with an accuracy of approximately 85.67%. The results indicate the potential of this model for sentiment analysis tasks.